# Instruct tuning the model

This notebook draws heavily a similar one done for the [phi3](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct/blob/main/sample_finetune.py) model. 

The difference here is that this will focus on a model's full fine-tuning process, work for going from a base model to a new insruction model, and should work for almost any model on HuggingFace.

At the end of the notebook are the steps to save this as a gguf format which will allow for fast and easy inference.

In [2]:
import sys
import logging

import datasets
from datasets import load_dataset
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import os
import json
import wandb


In [3]:
logger = logging.getLogger(__name__)
wandb.init(project="smollm-ft")
###################
# Hyper-parameters
###################
training_config = {
    "do_eval": False,
    "learning_rate": 2.0e-4,
    "weight_decay": 0.001,
    "per_device_train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "log_level": "info",
    "logging_steps": 150,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 8,
    "max_steps": -1,
    "output_dir": "./picard-ft",
    "overwrite_output_dir": True,
    "remove_unused_columns": True,
    "save_steps": 500,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.05,
    "report_to":"wandb",
    "neftune_noise_alpha":5,
    "push_to_hub": True,
    }

train_conf = TrainingArguments(**training_config)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: noahpunintended. Use `wandb login --relogin` to force relogin


In [4]:
###############
# Setup logging
###############
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

logger.info(f"Training/evaluation parameters {train_conf}")


2024-11-13 12:25:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_str

In [5]:


####################
# Base Model Loading
####################
checkpoint_path = "HuggingFaceTB/SmolLM2-360M"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
#    attn_implementation="flash_attention_2",  # only works on latest gpus, probably not worth it in most cases
     torch_dtype=torch.bfloat16,
   device_map='auto'
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)


###################
# Tokenizer Loading
###################

checkpoint_path = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = "<|endoftext|>"  # note this is specific to smollm
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'
# https://stackoverflow.com/questions/76446228/setting-padding-token-as-eos-token-when-using-datacollatorforlanguagemodeling-fr


[INFO|configuration_utils.py:733] 2024-11-13 12:25:09,201 >> loading configuration file config.json from cache at /home/zeus/.cache/huggingface/hub/models--HuggingFaceTB--SmolLM2-360M/snapshots/3ce05f63c246c44616da500b47b01f082f4d3bcc/config.json
[INFO|configuration_utils.py:800] 2024-11-13 12:25:09,204 >> Model config LlamaConfig {
  "_name_or_path": "HuggingFaceTB/SmolLM2-360M",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "silu",
  "hidden_size": 960,
  "initializer_range": 0.02,
  "intermediate_size": 2560,
  "is_llama_config": true,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 15,
  "num_hidden_layers": 32,
  "num_key_value_heads": 5,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_interleaved": false,
  "rope_scaling": null,
  "rope_theta": 100000,
  "tie_word_embeddings": true,
  "torch_dtype": "

In [6]:
tokenizer.pad_token, tokenizer.eos_token, tokenizer.eos_token_id, tokenizer.pad_token_id

('<|endoftext|>', '<|im_end|>', 2, 0)

### Setting up the fine-tune 

Now that the synthetic dataset is made, next up is ensure the model is capable of answering like we expect, without the large system prompt impacting latency. 

The solution to this is to open up the dataset, replace the system prompt with something much simpler, and starting training with that.

In [7]:
##################
# Data Processing
##################
def apply_chat_template(
    example,
    tokenizer,
):
    messages = example["messages"]['messages']
    messages[0]['content'] = "You are Pi-Card, the Raspberry Pi AI Assistant."
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False).strip('\n')
    return example

raw_dataset = load_dataset("nkasmanoff/pi-card-sft-data") 


train_dataset = raw_dataset["train"]

processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    desc="Applying chat template to train_sft",
)

# shuffle the dataset
processed_train_dataset = processed_train_dataset.shuffle(seed=42)

Overwrite dataset info from restored data version if exists.


2024-11-13 12:25:16 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.


Loading Dataset info from /home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6


2024-11-13 12:25:16 - INFO - datasets.info - Loading Dataset info from /home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6


Found cached dataset pi-card-sft-data (/home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6)


...

2024-11-13 12:25:21 - INFO - datasets.arrow_dataset - Loading cached processed dataset at /home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6/cache-7b2331d0b396f378_*_of_00010.arrow


Concatenating 10 shards


2024-11-13 12:25:21 - INFO - datasets.arrow_dataset - Concatenating 10 shards


Loading cached shuffled indices for dataset at /home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6/cache-6b02c55b1008bc29.arrow


2024-11-13 12:25:21 - INFO - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6/cache-6b02c55b1008bc29.arrow


In [8]:
processed_train_dataset[123]

{'messages': {'messages': [{'content': 'You are Pi-Card, the Raspberry Pi AI Assistant.',
    'role': 'system'},
   {'content': 'How does Joel Salzberg\'s book, "Critical Essays on Bernard Malamud," contribute to the understanding of Malamud\'s work in the context of American literature?: Essays and criticism on Bernard Malamud\'s The Natural - Critical Essays.\nEssays and criticism on Bernard Malamud - Malamud, Bernard.\nAmazon.com: Critical Essays on Bernard Malamud (Critical Essays on American Literature) (9780816188819): Joel Salzberg: Books.\nThe Natural by Bernard Malamud essaysIn his novel, "The Natural", Bernard Malamud presents a paradox of a character in Roy Hobbs. Roy is a talented athlete who fails to live up to the world\'s expectations of him despite his good intentions. For all of his physical tools, Roy cannot overcome his.',
    'role': 'user'},
   {'content': 'Joel Salzberg\'s book, "Critical Essays on Bernard Malamud," provides in-depth analysis and insights into Mal

In [9]:
model.eval();
system_prompt = "You are Pi-Card, the Raspberry Pi AI Assistant."
#prompt = """What is the oort cloud?"""
prompt = "I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?"
#prompt = f"<|im_start|>system\nYou are Pi-CARD, the Raspberry Pi AI assistant<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:2]) + "<|im_end|>"
print(formatted_output_text)

<|im_start|>user
I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?<|im_end|>
<|im_start|>assistant
I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wilderness survival trip and I'm not sure how to find food. Can you help me?

I'm going on a wildern

In [10]:
###########
# Training
###########

model.train();
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    train_dataset=processed_train_dataset,
    max_seq_length=2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    #packing=True,
)
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

# trainer.push_to_hub()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
[INFO|training_args.py:2100] 2024-11-13 12:25:39,675 >> PyTorch: setting up devices
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Loading cach

2024-11-13 12:25:39 - INFO - datasets.arrow_dataset - Loading cached processed dataset at /home/zeus/.cache/huggingface/datasets/nkasmanoff___pi-card-sft-data/default/0.0.0/ace7c218bc809b351ad35add013fe25169b7c2d6/cache-3d17a443d5786dd7.arrow


[INFO|trainer.py:2134] 2024-11-13 12:25:40,556 >> ***** Running training *****
[INFO|trainer.py:2135] 2024-11-13 12:25:40,557 >>   Num examples = 5,660
[INFO|trainer.py:2136] 2024-11-13 12:25:40,557 >>   Num Epochs = 8
[INFO|trainer.py:2137] 2024-11-13 12:25:40,558 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:2140] 2024-11-13 12:25:40,559 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:2141] 2024-11-13 12:25:40,559 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2142] 2024-11-13 12:25:40,560 >>   Total optimization steps = 5,664
[INFO|trainer.py:2143] 2024-11-13 12:25:40,561 >>   Number of trainable parameters = 361,821,120
[INFO|integration_utils.py:807] 2024-11-13 12:25:40,564 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a dif

Step,Training Loss
150,1.555100
300,1.171400
450,1.189600
600,1.130900
750,1.107400
900,1.004600
1050,1.002600
1200,0.974000
1350,1.025400
1500,0.902000


[INFO|trainer.py:3503] 2024-11-13 12:43:54,020 >> Saving model checkpoint to ./picard-ft/checkpoint-500
[INFO|configuration_utils.py:472] 2024-11-13 12:43:54,022 >> Configuration saved in ./picard-ft/checkpoint-500/config.json
[INFO|configuration_utils.py:807] 2024-11-13 12:43:54,023 >> Configuration saved in ./picard-ft/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:2799] 2024-11-13 12:43:55,323 >> Model weights saved in ./picard-ft/checkpoint-500/model.safetensors
[INFO|tokenization_utils_base.py:2684] 2024-11-13 12:43:55,325 >> tokenizer config file saved in ./picard-ft/checkpoint-500/tokenizer_config.json
[INFO|tokenization_utils_base.py:2693] 2024-11-13 12:43:55,326 >> Special tokens file saved in ./picard-ft/checkpoint-500/special_tokens_map.json
[INFO|tokenization_utils_base.py:2684] 2024-11-13 12:43:58,064 >> tokenizer config file saved in ./picard-ft/tokenizer_config.json
[INFO|tokenization_utils_base.py:2693] 2024-11-13 12:43:58,065 >> Special tokens file saved

***** train metrics *****
  epoch                    =         8.0
  total_flos               = 121992911GF
  train_loss               =      0.8536
  train_runtime            =  3:27:14.26
  train_samples_per_second =       3.642
  train_steps_per_second   =       0.456


# Evaluation and saving the model

In [ ]:
# Load the model from the checkpoint

# find most recently created folder in checkpoint_dir and set as checkpoint path
checkpoint_path = sorted(os.listdir(train_conf.output_dir))[-1]
checkpoint_path = os.path.join(train_conf.output_dir, checkpoint_path)
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
     torch_dtype=torch.bfloat16,
   device_map='auto'
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)


In [18]:
model.eval();
prompt = """Who is George RR Martin?"""
prompt = f"<|im_start|>system\nYou are Pi-Card, the Raspberry Pi AI Assistant.<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>" 
print(formatted_output_text)

<|im_start|>system
You are Pi-Card, the Raspberry Pi AI Assistant.<|im_end|>
<|im_start|>user
Who is George RR Martin?<|im_end|>
<|im_start|>assistant
George R.R. Martin is a renowned American author, best known for his epic fantasy series "A Song of Ice and Fire," which has become a cultural phenomenon.<|im_end|>


# Saving to gguf
#https://github.com/ggerganov/llama.cpp/discussions/2948




In [ ]:
# Start by downloading llama-cpp if not already done

#!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt

In [ ]:
# Create gguf file

# Please note you'll need to update the checkpoint path and model names to the one you want to convert & save
!python llama.cpp/convert_hf_to_gguf.py nature-buddy/checkpoint-2005 --outfile nature-buddy-0.135b-f16.gguf --outtype f16


The quanitzation output is going to have an outsized impact on latency / performance. 

While f16 is the default and good, it's worth noting the model was trained using bf16, a slightly different format, so that outtype may be worth testing.

Now that you have the gguf you can either work with that directly, or convert it to an ollama format, which can be easier to work with in some cases. 

For instructions on how to do this, please see the instructions in create ollama text file.